<a href="https://colab.research.google.com/github/aclaragonzalez/ABCData/blob/main/json_gaussian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Descrição do projeto
O projeto consiste em uma extração de propriedades químicas de compostos a partir de arquivos de saída de dois softwares (Gaussian e Orca) utilizados na Química Computacional. A extração dessas propriedades visa reunir dados em uma base hospedada no MongoDB.

## Descrição dos arquivos de saída
Os arquivos de saída dos softwares possuem formato .log ou .out. Já existe uma biblioteca, cclib, que extrai as propriedades químicas desses softwares. A biblioteca será utilizada no terminal para obter arquivos .json com os dados que serão limpos e organizados neste notebook.

No terminal, o seguinte comando foi utilizado para arquivos .log:
*   ccwrite json *.log

Para arquivos .out:
*   ccwrite json *.out


## Código implementado

In [ ]:
import json
import cclib
import uuid
from os import listdir
from os.path import isfile, join, splitext
import subprocess
from subprocess import PIPE

# user se refere ao autor do cálculo, class é a classe da molécula e solvent é o solvente utilizado no cálculo

path = './user/solvent/class'

id_list = []

files = [f for f in listdir(path) if isfile(join(path, f))]

for fname in files:
    if fname.endswith('.log') or fname.endswith('.out'):        
        with open(join(path,fname)) as f:
            data = cclib.io.ccread(f)
        
        filename = splitext(fname)[0]
        filename = filename + '.json'
        
        with open(join(path,filename)) as file:
            # carrega o arquivo json
            jdata = json.load(file)
            
            # adiciona um id para cada arquivo json
            digits = 8
            real_id = uuid.uuid4().int
            str_id = str(real_id)
            simple_id = str_id[:digits]
            if simple_id in id_list:
                real_id = uuid.uuid4().int
                str_id = str(real_id)
                simple_id = str_id[:digits]
                jdata['_id'] = simple_id
            else:
                jdata['_id'] = simple_id
                
            # adiciona simple_id ao name
            jdata['name'] = simple_id

            # altera as coordenadas para uma matriz Nx3
            coords = jdata['atoms']['coords']['3d']
            jdata['atoms']['coords-3d'] = [coords[i:i+3] for i in range(0,len(coords),3)]
            del jdata['atoms']['coords']

            # deleta a key core electrons
            del jdata['atoms']['core electrons']
            
            # tira o número de átomos da key atoms
            atomcount = jdata['atoms']['elements']['atom count']
            jdata['atom_count'] = atomcount
            del jdata['atoms']['elements']['atom count']
            
            # tira o número de átomos pesados da key atoms
            h_atomcount = jdata['atoms']['elements']['heavy atom count']
            jdata['heavy_atom_count'] = h_atomcount
            del jdata['atoms']['elements']['heavy atom count']
            
            # mudando o nome de uma key: adiciona a nova key com os valores da velha e apaga a velha
            jdata['atoms']['elements']['atomic number'] = jdata['atoms']['elements']['number']
            del jdata['atoms']['elements']['number']
            
            # muda a key momento de dipolo de lugar
            dipole = jdata['properties']['total dipole moment']
            jdata['dipole_moment'] = dipole
            
            # muda a key cargas de lugar
            mulliken = jdata['properties']['partial charges']['mulliken']
            jdata['partial charges'] = {'mulliken': mulliken}
            
            # muda a key orbitals de posição
            orbitals = jdata['properties']['orbitals']
            jdata['orbitals'] = orbitals
            
            #adiciona dados de energia
            E_homo = float(data.moenergies[0][data.homos])
            E_lumo = float(data.moenergies[0][data.homos + 1])
            E_gap = E_homo - E_lumo
            total_energy = jdata['properties']['energy']['total']
            jdata['orbitals']['homo'] = E_homo
            jdata['orbitals']['lumo'] = E_lumo
            jdata['orbitals']['gap'] = E_gap
            jdata['energy'] = {'total': total_energy}
            
            # adiciona dados do cálculo
            info_calculation = data.metadata
            method = info_calculation['methods'][0]
            info_calculation['method'] = method
            del info_calculation['methods']
            jdata['calculation info'] = data.metadata
                            
            # adiciona dados extras
            jdata['class'] = 'porphyrin'
            jdata['authors'] = 'Ana Clara Gonzalez de Souza e Paula Homem de Mello'
            jdata['calculation info']['solvent'] = 'tetrahydrofuran'
            jdata['doi'] = 'somente para moléculas publicadas'
            
            # organizando properties
            charge = jdata['properties']['charge']
            jdata['calculation info']['charge'] = charge
            
            multiplicity = jdata['properties']['multiplicity']
            jdata['calculation info']['multiplicity'] = multiplicity
            
            del jdata['properties']

            # calculo do comprimento de onda referente às transições eletrônicas
            nm_list = []

            for i in jdata['transitions']['electronic transitions']:
                nm = (1.98644582 * 10**-25)/(i*1.98630*10**-23)*10**9
                nm = round(nm,2)
                nm_list.append(nm)
                jdata['transitions']['wavelengths'] = nm_list
        
        # atualiza todas as mudanças no arquivo json    
        with open(join(path,filename), "w") as file:
            json.dump(jdata, file, indent=4, sort_keys=True, default=str)